In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Load the dataset from CSV
df = pd.read_csv('reviews.csv')  # Ensure the CSV has 'Sentence' and 'Aspects' columns

# Extract columns
sentences = df['Sentence'].tolist()
aspects = df['Aspects'].tolist()

# Assign binary labels (e.g., 0 for negative, 1 for positive). Adjust this as per your dataset's labeling.
labels = [1 if "good" in sentence.lower() else 0 for sentence in sentences]

# Function to preprocess the data
def preprocess(sentences, aspects, labels):
    input_ids = []
    attention_masks = []
    label_list = []

    for sentence, aspect, label in zip(sentences, aspects, labels):
        encoded = tokenizer(
            f"{sentence} [SEP] {aspect}",
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'].squeeze(0))
        attention_masks.append(encoded['attention_mask'].squeeze(0))
        label_list.append(label)

    return torch.stack(input_ids), torch.stack(attention_masks), torch.tensor(label_list)

input_ids, attention_masks, labels = preprocess(sentences, aspects, labels)

# Create DataLoader
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Load the pre-trained XLNet model
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)
model.to(device)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(dataloader) * 4  # Assume 4 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
train_losses = []
val_losses = []
epochs = 4
for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    for batch in dataloader:
        batch_input_ids, batch_attention_masks, batch_labels = [x.to(device) for x in batch]

        model.zero_grad()
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(dataloader)
    train_losses.append(avg_train_loss)

    # Validation phase (using the same DataLoader for simplicity; ideally, use a separate validation set)
    model.eval()
    total_val_loss = 0
    val_predictions, val_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            batch_input_ids, batch_attention_masks, batch_labels = [x.to(device) for x in batch]
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            val_predictions.extend(predictions)
            val_labels.extend(batch_labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(dataloader)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.show()

# Confusion Matrix and Metrics
conf_matrix = confusion_matrix(val_labels, val_predictions)
accuracy = accuracy_score(val_labels, val_predictions)
precision = precision_score(val_labels, val_predictions, average='weighted')
recall = recall_score(val_labels, val_predictions, average='weighted')
f1 = f1_score(val_labels, val_predictions, average='weighted')

# F2 Score calculation
def fbeta_score(y_true, y_pred, beta):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    return (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)

f2 = fbeta_score(val_labels, val_predictions, beta=2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"F2 Score: {f2:.4f}")

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()
